In [42]:
from enum import StrEnum, auto
import json

from dotenv import load_dotenv
from pydantic import BaseModel, Field, HttpUrl
import requests

from wow_api.external.blizz_api.auth import BlizzardApiAuth

load_dotenv()

True

In [5]:
class BlizzardApiRegion(StrEnum):
    us = auto()
    eu = auto()
    kr = auto()
    tw = auto()


class BlizzardNamespaceCategory(StrEnum):
    static = auto()
    dynamic = auto()
    profile = auto()


class BlizzardApiLocales(StrEnum):
    en_us = "en_US"
    es_mx = "es_MX"
    pt_br = "pt_BR"
    de_de = "de_DE"
    en_gb = "en_GB"
    es_es = "es_ES"
    fr_fr = "fr_FR"
    it_it = "it_IT"
    ru_RU = "ru_RU"
    ko_kr = "ko_KR"
    zh_tw = "zh_TW"
    zh_cn = "zh_CN"



In [3]:
api_auth = BlizzardApiAuth()
token = api_auth.get_token()

made new token: access_token='USOG7QkKNVolljRksvNfnyD9HL6SoQrQ97' token_type='bearer' expires_in=datetime.timedelta(seconds=86399) sub='e99ce1a2c0454313b5f5b95d5cce9901' expires_at=datetime.datetime(2023, 4, 5, 18, 2, 45, 297083, tzinfo=datetime.timezone.utc)
access_token='USOG7QkKNVolljRksvNfnyD9HL6SoQrQ97' token_type='bearer' expires_in=datetime.timedelta(seconds=86399) sub='e99ce1a2c0454313b5f5b95d5cce9901' expires_at=datetime.datetime(2023, 4, 5, 18, 2, 45, 297083, tzinfo=datetime.timezone.utc)


In [39]:
default_region: BlizzardApiRegion = BlizzardApiRegion.us
default_locale: BlizzardApiLocales = BlizzardApiLocales.en_us

default_params = {
        'locale': default_locale.value
    }


def build_url(endpoint: str, region: BlizzardApiRegion = default_region) -> str:
    return f"https://{region.value}.api.blizzard.com{endpoint}"


def build_headers(
    namespace_category: BlizzardNamespaceCategory | None = None,
    region: BlizzardApiRegion = default_region,
) -> dict[str, str]:
    if namespace_category:
        return {
            "Authorization": f"Bearer {token.access_token}",
            "Battlenet-Namespace": f"{namespace_category.value}-{region.value}",
        }

    else:
        return {
            "Authorization": f"Bearer {token.access_token}",
        }
    
def get_url_params(url: HttpUrl) -> dict[str, str]:
    query_string = url.query
    query_params = query_string.split("&")
    print(query_params)

    param_dict = {}
    for p in query_params:
        key_value = p.split("=")
        if len(key_value) > 0:
            param_dict[key_value[0]] = key_value[1]

    return param_dict

In [10]:
class Self(BaseModel):
    href: HttpUrl


class FieldLinks(BaseModel):
    self: Self


class ConnectedRealm(BaseModel):
    href: HttpUrl


class ConnectedRealmsListResponse(BaseModel):
    field_links: FieldLinks = Field(..., alias="_links")
    connected_realms: list[ConnectedRealm]

def get_connected_realms_list(locale: BlizzardApiLocales = default_locale) -> ConnectedRealmsListResponse:
    url = build_url("/data/wow/connected-realm/index")
    
    connected_realms_res = requests.get(url, headers=build_headers(BlizzardNamespaceCategory.dynamic), params={
        'locale': locale.value
    })

    return ConnectedRealmsListResponse.parse_obj(connected_realms_res.json())


In [46]:
class Status(BaseModel):
    type: str
    name: str


class Population(BaseModel):
    type: str
    name: str


class Key(BaseModel):
    href: HttpUrl


class Region(BaseModel):
    key: Key
    name: str
    id: int


class Type(BaseModel):
    type: str
    name: str


class Realm(BaseModel):
    id: int
    region: Region
    connected_realm: ConnectedRealm
    name: str
    category: str
    locale: str
    timezone: str
    type: Type
    is_tournament: bool
    slug: str


class MythicLeaderboards(BaseModel):
    href: HttpUrl


class Auctions(BaseModel):
    href: HttpUrl


class ConnectedRealmResponse(BaseModel):
    field_links: FieldLinks = Field(..., alias='_links')
    id: int
    has_queue: bool
    status: Status
    population: Population
    realms: list[Realm]
    mythic_leaderboards: MythicLeaderboards
    auctions: Auctions

def get_connected_realm() -> None:
    pass

In [11]:
connected_realms_list = get_connected_realms_list()

In [44]:
connected_realm_res = requests.get(connected_realms_list.connected_realms[0].href, headers=build_headers(), params=default_params)

In [45]:
with open('./out/raw.json', 'w') as file:
    json.dump(connected_realm_res.json(), file, indent=4)

In [47]:
connected_realm = ConnectedRealmResponse.parse_obj(connected_realm_res.json())

In [49]:
mythic_leaderboards_res = requests.get(
        connected_realm.mythic_leaderboards.href,
        headers=build_headers(),
        params=default_params
    )


In [53]:
current_leaderboards = RealmCurrentLeaderboardsResponse.parse_obj(mythic_leaderboards_res.json())

In [56]:
l = current_leaderboards.current_leaderboards[0]

In [57]:
leaderboard_req = requests.get(
            l.key.href,
            headers=build_headers(),
            params=default_params
        )

In [58]:
with open('./out/raw.json', 'w') as file:
    json.dump(leaderboard_req.json(), file, indent=4)

In [79]:
leaderboard_req.

<PreparedRequest [GET]>

In [71]:
leaderboard = LeaderboardResponse.parse_obj(leaderboard_req.json())

In [63]:
dungeon_name = leaderboard.name.lower().replace(" ", "-")

In [64]:
dungeon_name

'temple-of-the-jade-serpent'

In [67]:
file_name = f"./out/{connected_realm.id}-{dungeon_name}-period-{leaderboard.period}.json",

In [69]:
len(leaderboard.leading_groups)

8